<a href="https://colab.research.google.com/github/pulum03/ML/blob/master/faster-rcnn_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setting

In [0]:
# pytorch 1.0.0 버전으로 설치하고, 
# git clone 해서 모델  옮기고, 그외 requirements 들 충족하도록 설치 (trello에 있음)

In [6]:
!pip install -q torch==1.0.0 torchvision==0.2.1

     |████████████████████████████████| 61kB 3.5MB/s 


In [3]:
!git clone -b pytorch-1.0 https://github.com/jwyang/faster-rcnn.pytorch.git

Cloning into 'faster-rcnn.pytorch'...
remote: Enumerating objects: 3846, done.
remote: Total 3846 (delta 0), reused 0 (delta 0), pack-reused 3846
Receiving objects: 100% (3846/3846), 6.21 MiB | 8.71 MiB/s, done.
Resolving deltas: 100% (2608/2608), done.


In [7]:
!pip list

Package                  Version    
------------------------ -----------
absl-py                  0.9.0      
alabaster                0.7.12     
albumentations           0.1.12     
altair                   4.0.0      
asgiref                  3.2.3      
astor                    0.8.1      
astropy                  4.0        
atari-py                 0.2.6      
atomicwrites             1.3.0      
attrs                    19.3.0     
audioread                2.1.8      
autograd                 1.3        
Babel                    2.8.0      
backcall                 0.1.0      
backports.tempfile       1.0        
backports.weakref        1.0.post1  
beautifulsoup4           4.6.3      
bleach                   3.1.0      
blis                     0.2.4      
bokeh                    1.4.0      
boto                     2.49.0     
boto3                    1.10.47    
botocore                 1.13.47    
Bottleneck               1.3.1      
branca                   0.3.1      
b

In [0]:
#!git clone https://github.com/jwyang/faster-rcnn.pytorch.git

fatal: destination path 'faster-rcnn.pytorch' already exists and is not an empty directory.


In [0]:
#!rm -rf faster-rcnn.pytorch

In [8]:
#!git branch

fatal: not a git repository (or any of the parent directories): .git


In [9]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [10]:
import torch
print(torch.__version__)

1.0.0


#Data Preparation

In [11]:
!pip install https://github.com/fastai/fastai/archive/master.zip
!pip install opencv-python
!apt update && apt install -y libsm6 libxext6
!mkdir data
!wget http://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar -P data/
!wget https://storage.googleapis.com/coco-dataset/external/PASCAL_VOC.zip -P data/
!tar -xf data/VOCtrainval_06-Nov-2007.tar -C data/
!unzip data/PASCAL_VOC.zip -d data/
!rm -rf data/PASCAL_VOC.zip data/VOCtrainval_06-Nov-2007.tar

     - 208.4MB 233kB/s
  Created wheel for fastai: filename=fastai-1.0.61.dev0-cp36-none-any.whl size=237453 sha256=e5b34f1a0de7a5b524ca223cca97b6c3042aef0f90ba8a2095cc210488a2617f
  Stored in directory: /tmp/pip-ephem-wheel-cache-4qhh1yn8/wheels/64/3d/9f/d12a217aa2531321c5b9ae96288fcae2687d3b744376e8f94f
Successfully built fastai
  Found existing installation: fastai 1.0.60
    Uninstalling fastai-1.0.60:
      Successfully uninstalled fastai-1.0.60
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:8 https://deve

Download pre-trained model

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [46]:
!wget "https://www.dropbox.com/s/iev3tkbz5wyyuz9/resnet101_caffe.pth"

--2020-01-10 08:31:32--  https://www.dropbox.com/s/iev3tkbz5wyyuz9/resnet101_caffe.pth
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/iev3tkbz5wyyuz9/resnet101_caffe.pth [following]
--2020-01-10 08:31:33--  https://www.dropbox.com/s/raw/iev3tkbz5wyyuz9/resnet101_caffe.pth
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc14feffcd3e1204b78d5cf9ac08.dl.dropboxusercontent.com/cd/0/inline/Av54kVYRTHHDc2Mm6UkAm9dSPuqCD2z0hRAExDRo_cnKPRUc2DZMfkYLc3iLLY97kt83uavT1NsAc2vmOiNiEikkq-7M6aavT3wz83kkSBEFhQ/file# [following]
--2020-01-10 08:31:33--  https://uc14feffcd3e1204b78d5cf9ac08.dl.dropboxusercontent.com/cd/0/inline/Av54kVYRTHHDc2Mm6UkAm9dSPuqCD2z0hRAExDRo_cnKPRUc2DZMfkYLc3iLLY97kt83uavT1NsAc2vmOiNiEikkq-7M6aavT3wz83kkSBEFhQ/fi

#Compilation

In [0]:
#!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [12]:
%cd faster-rcnn.pytorch/lib
!ls

/content/faster-rcnn.pytorch/lib
datasets  model  pycocotools  roi_data_layer  setup.py


In [13]:
!python setup.py build develop

running build
running build_py
creating build
creating build/lib.linux-x86_64-3.6
creating build/lib.linux-x86_64-3.6/datasets
copying datasets/coco.py -> build/lib.linux-x86_64-3.6/datasets
copying datasets/pascal_voc_rbg.py -> build/lib.linux-x86_64-3.6/datasets
copying datasets/ds_utils.py -> build/lib.linux-x86_64-3.6/datasets
copying datasets/vg_eval.py -> build/lib.linux-x86_64-3.6/datasets
copying datasets/imdb.py -> build/lib.linux-x86_64-3.6/datasets
copying datasets/imagenet.py -> build/lib.linux-x86_64-3.6/datasets
copying datasets/pascal_voc.py -> build/lib.linux-x86_64-3.6/datasets
copying datasets/__init__.py -> build/lib.linux-x86_64-3.6/datasets
copying datasets/factory.py -> build/lib.linux-x86_64-3.6/datasets
copying datasets/voc_eval.py -> build/lib.linux-x86_64-3.6/datasets
copying datasets/vg.py -> build/lib.linux-x86_64-3.6/datasets
creating build/lib.linux-x86_64-3.6/model
copying model/__init__.py -> build/lib.linux-x86_64-3.6/model
creating build/lib.linux-x86_

In [35]:
!pip uninstall scipy
!pip install scipy==1.1.0

Uninstalling scipy-1.4.1:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/scipy-1.4.1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/scipy/*
Proceed (y/n)? y
  Successfully uninstalled scipy-1.4.1
     |████████████████████████████████| 31.2MB 97kB/s 
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [14]:
!ls

build	  faster_rcnn.egg-info	pycocotools	setup.py
datasets  model			roi_data_layer


#Train

In [0]:
import os
os.chdir("/content/faster-rcnn.pytorch")
!pwd

# Change the folder name 'VOCdevkit' to 'VOCdevkit2007' in faster-rcnn.pytorch/data

In [0]:
# vgg 16
# !CUDA_VISIBLE_DEVICES=0 python lib/trainval_net.py \
#                    --dataset pascal_voc --net vgg16 \
#                    --bs 16 --nw 12 \
#                    --lr 0.001 --lr_decay_step 5 \
#                    --cuda

python3: can't open file 'trainval_net.py': [Errno 2] No such file or directory


In [44]:
# pascal_vod
!CUDA_VISIBLE_DEVICES=0 python trainval_net.py \
                   --dataset pascal_voc --net res101 \
                   --bs 4 --nw 12 \
                   --lr 0.001 --lr_decay_step 5 \
                   --cuda

[session 1][epoch  1][iter  100/2505] loss: 1.7132, lr: 1.00e-03
			fg/bg=(109/403), time cost: 270.678101
			rpn_cls: 0.2013, rpn_box: 0.0785, rcnn_cls: 0.6256, rcnn_box 0.5032
[session 1][epoch  1][iter  200/2505] loss: 1.3392, lr: 1.00e-03
			fg/bg=(105/407), time cost: 269.177291
			rpn_cls: 0.1757, rpn_box: 0.1010, rcnn_cls: 0.4376, rcnn_box 0.5081
Traceback (most recent call last):
  File "trainval_net.py", line 312, in <module>
    im_data.resize_(data[0].size()).copy_(data[0])
KeyboardInterrupt


KeyboardInterrupt: ignored

In [0]:
# python trainval_net.py --dataset pascal_voc --net vgg16 \
#                        --bs 24 --nw 8 \
#                        --lr $LEARNING_RATE --lr_decay_step $DECAY_STEP \
#                        --cuda --mGPUs

#Test

In [0]:
python demo.py --net res101 \
               --checksession 1 --checkepoch 2 --checkpoint 2504 \
               --cuda --load_dir /models